<a href="https://colab.research.google.com/github/Chiamakac/IgboNER-Models/blob/main/Model/IgboBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Training an Igbo language model from scratch using Transformers and Tokenizers**

**1. Getting the data.**

In [ ]:
!wget -c https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
!wget -c https://raw.githubusercontent.com/chiamaka249/lacuna_pos_ner/main/language_corpus/ibo/ibo.txt
!wget -c https://raw.githubusercontent.com/Chiamakac/IboBETA/main/config.json

In [ ]:
#Unzip the zipped file and remove the zipped file after unzipping
import zipfile
import os


def unzip(zipfilename):
  try:
    with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
      zip_ref.extractall(zipfilename[:-4])
      return f"'{zipfilename}' unzipped!"
  except FileNotFoundError:
    print(f"Cannot find '{zipfilename}' file")

unzip("text.zip")
!rm text.zip

In [ ]:
#copies the file "ibo.txt" into the folder "text"
import shutil
shutil.move('/content/ibo.txt', '/content/text')



'/content/text/ibo.txt'

In [ ]:

# import os
#import shutil
dir_name = "/content/text"
text=""
for fname in os.listdir(dir_name):
  fname = os.path.join(dir_name, fname)
  with open(fname, "r", encoding="utf8") as datafile:
    text = text+"\n"+datafile.read()

with open("data.txt", "w", encoding="utf8") as datafile:
  datafile.write(text)

shutil.rmtree("text")

**2.  Import Transformers, Tokenizer and Train the tokenizer**

In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow

# Install `transformers` from master stating the version
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

In [ ]:

%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

# Describing the path to all of our Igbo data 
paths = [str(x) for x in Path(".").glob("**/*.txt")]
print(paths)

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",#Beginning of sequence (BOS) or classifier (CLS) token
    "<pad>",# Padding token
    "</s>",#End of sequence (EOS) or seperator (SEP) token
    "<unk>",# Unknown token
    "<mask>", # Masking token
])

['data.txt']
CPU times: user 18.9 s, sys: 1.2 s, total: 20.1 s
Wall time: 6.07 s


In [ ]:
#Our tokenizer is now ready and we have two files that define our new IgboBert tokenizer( a vocab.json-which is a list of the most frequent tokens ranked by frequency and a merges.txt list of merges)
#we then save the file for later use

!mkdir IgboBert
tokenizer.save_model("IgboBert")

['IgboBert/vocab.json', 'IgboBert/merges.txt']

In [ ]:
shutil.move('/content/config.json', '/content/IgboBert')

'/content/IgboBert/config.json'

**2. Initializing the Tokenizer**

Let's initialize our tokenizer. This way we can use it as we would use any other from_pretrained tokenizer.

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./IgboBert/vocab.json",
    "./IgboBert/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Simone gara ụka ụnyahụ guọ egwu ma ga-kwa taa.", "Aha ya bụ ifeoma.").tokens

['<s>',
 'Simone',
 'Ġgara',
 'Ġá»¥ka',
 'Ġá»¥nyahá»¥',
 'Ġgu',
 'á»į',
 'Ġegwu',
 'Ġma',
 'Ġga',
 '-',
 'kwa',
 'Ġtaa',
 '.',
 '</s>',
 'Aha',
 'Ġya',
 'Ġbá»¥',
 'Ġife',
 'oma',
 '.',
 '</s>']

In [ ]:
# Check that we have a GPU
!nvidia-smi

Wed Dec 15 20:56:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
#For training, we need a raw (not pre-trained) BERTLMHeadModel. 
#To create that, we first need to create a RoBERTa config object to describe the parameters we’d like to initialize IgboBERT with.

from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)


In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./IgboBert", max_len=512, config=config)

In [ ]:
#We import and initialize our RoBERTa model with a language modeling (LM) head.

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 83 million parameters

83504416

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "/content/data.txt",
    block_size = 128
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 33.3 s, sys: 1.27 s, total: 34.5 s
Wall time: 15.6 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./IgboBert",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
%%time
trainer.train()

In [ ]:
trainer.save_model("./IgboBert")

Saving model checkpoint to ./IgboBert
Configuration saved in ./IgboBert/config.json
Model weights saved in ./IgboBert/pytorch_model.bin


# **4. Test the Model**

We first initialize a pipeline object, using the 'fill-mask' argument. Then begin testing our model like so



In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./IgboBert",
    tokenizer="./IgboBert"
)

In [ ]:
# The sun <mask>.
# =>

fill_mask("Abụ m Maazị <mask>.") #= okafor/Ọkafọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.008856686763465405,
  'sequence': 'Abụ m Maazị Mohammed.',
  'token': 3231,
  'token_str': ' Mohammed'},
 {'score': 0.007911812514066696,
  'sequence': 'Abụ m Maazị O.',
  'token': 381,
  'token_str': ' O'},
 {'score': 0.00748562254011631,
  'sequence': 'Abụ m Maazị Ọkafọ.',
  'token': 5307,
  'token_str': ' Ọkafọ'},
 {'score': 0.006802904885262251,
  'sequence': 'Abụ m Maazị A.',
  'token': 348,
  'token_str': ' A'},
 {'score': 0.0032701685559004545,
  'sequence': 'Abụ m Maazị Rutherford.',
  'token': 5113,
  'token_str': ' Rutherford'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.1661785989999771,
  'sequence': 'Nwaanyị na ya ji na akara.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.11589646339416504,
  'sequence': 'Nwaanyị na nwaanyị ji na akara.',
  'token': 623,
  'token_str': ' nwaanyị'},
 {'score': 0.03902192786335945,
  'sequence': 'Nwaanyị na nwunye ji na akara.',
  'token': 724,
  'token_str': ' nwunye'},
 {'score': 0.038711633533239365,
  'sequence': 'Nwaanyị na nna ji na akara.',
  'token': 713,
  'token_str': ' nna'},
 {'score': 0.02920209988951683,
  'sequence': 'Nwaanyị na- ji na akara.',
  'token': 17,
  'token_str': '-'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Chineke ga- ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.24352985620498657,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọma.',
  'token': 496,
  'token_str': ' ọma'},
 {'score': 0.16800811886787415,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọjọọ.',
  'token': 707,
  'token_str': ' ọjọọ'},
 {'score': 0.15601330995559692,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe a.',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.11959126591682434,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe niile.',
  'token': 427,
  'token_str': ' niile'},
 {'score': 0.02550322934985161,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe oriri.',
  'token': 1580,
  'token_str': ' oriri'}]

In [ ]:
fill_mask("ọba akwụkwọ Ọkammụta Kenneth Dike dị <mask>.") #n'Awka

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.10755623877048492,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị mkpa.',
  'token': 607,
  'token_str': ' mkpa'},
 {'score': 0.07236427068710327,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị nso.',
  'token': 604,
  'token_str': ' nso'},
 {'score': 0.067164845764637,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị mma.',
  'token': 347,
  'token_str': ' mma'},
 {'score': 0.054604168981313705,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị iche.',
  'token': 462,
  'token_str': ' iche'},
 {'score': 0.04094060882925987,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị ukwuu.',
  'token': 1009,
  'token_str': ' ukwuu'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri <mask> na akara.") #= ji
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.12033308297395706,
  'sequence': 'Nwaanyị na eri ya na akara.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.07522733509540558,
  'sequence': 'Nwaanyị na eri nri na akara.',
  'token': 870,
  'token_str': ' nri'},
 {'score': 0.0471433624625206,
  'sequence': 'Nwaanyị na eri ihe na akara.',
  'token': 300,
  'token_str': ' ihe'},
 {'score': 0.02259232848882675,
  'sequence': 'Nwaanyị na eri eri na akara.',
  'token': 957,
  'token_str': ' eri'},
 {'score': 0.01788548193871975,
  'sequence': 'Nwaanyị na eri nwaanyị na akara.',
  'token': 623,
  'token_str': ' nwaanyị'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Gaanụ mee ndị <mask> niile ka ha bụrụ ndị na- eso ụzọ m  .") #= mba


[{'score': 0.3522748351097107,
  'sequence': 'Gaanụ mee ndị a niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.08272776752710342,
  'sequence': 'Gaanụ mee ndị ọzọ niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 434,
  'token_str': ' ọzọ'},
 {'score': 0.06121758744120598,
  'sequence': 'Gaanụ mee ndị ahụ niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 310,
  'token_str': ' ahụ'},
 {'score': 0.06120830774307251,
  'sequence': 'Gaanụ mee ndị mmadụ niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 393,
  'token_str': ' mmadụ'},
 {'score': 0.04012814536690712,
  'sequence': 'Gaanụ mee ndị Izrel niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 680,
  'token_str': ' Izrel'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ <mask>.") #= Izrel


[{'score': 0.3820941150188446,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ Izrel.',
  'token': 680,
  'token_str': ' Izrel'},
 {'score': 0.2837800979614258,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ ya.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.10724257677793503,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ mmadụ.',
  'token': 393,
  'token_str': ' mmadụ'},
 {'score': 0.024513551965355873,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ ha.',
  'token': 296,
  'token_str': ' ha'},
 {'score': 0.01360291987657547,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ Igbo.',
  'token': 900,
  'token_str': ' Igbo'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka <mask> Haram.") #= Boko


[{'score': 0.8887956738471985,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Boko Haram.',
  'token': 2535,
  'token_str': ' Boko'},
 {'score': 0.007407982833683491,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Manchester Haram.',
  'token': 3278,
  'token_str': ' Manchester'},
 {'score': 0.007324530277401209,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Super Haram.',
  'token': 3199,
  'token_str': ' Super'},
 {'score': 0.0033930952195078135,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka ịgba Haram.',
  'token': 874,
  'token_str': ' ịgba'},
 {'score': 0.0025665571447461843,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Gọvanọ Haram.',
  'token': 1692,
  'token_str': ' Gọvanọ'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado <mask> ọhụrụ a.") #= iwu


[{'score': 0.1165534108877182,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ọchịchị ọhụrụ a.',
  'token': 719,
  'token_str': ' ọchịchị'},
 {'score': 0.056705061346292496,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ọrụ ọhụrụ a.',
  'token': 477,
  'token_str': ' ọrụ'},
 {'score': 0.04727887734770775,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ọnọdụ ọhụrụ a.',
  'token': 1036,
  'token_str': ' ọnọdụ'},
 {'score': 0.042590655386447906,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ụwa ọhụrụ a.',
  'token': 556,
  'token_str': ' ụwa'},
 {'score': 0.0251015517860651,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado obodo ọhụrụ a.',
  'token': 576,
  'token_str': ' obodo'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask(" <mask> sị ka ehiwe ụlọikpe pụrụiche maka mpụ.") #= Buhari


[{'score': 0.44573667645454407,
  'sequence': 'A sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 37,
  'token_str': 'A'},
 {'score': 0.230531707406044,
  'sequence': 'Ọ sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 336,
  'token_str': 'Ọ'},
 {'score': 0.03158653527498245,
  'sequence': 'Ha sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 513,
  'token_str': 'Ha'},
 {'score': 0.02112211100757122,
  'sequence': 'Igbo sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 3656,
  'token_str': 'Igbo'},
 {'score': 0.011359370313584805,
  'sequence': 'O sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 51,
  'token_str': 'O'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Ala <mask>  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.") #= Naịjirịa


[{'score': 0.10420944541692734,
  'sequence': 'Ala a  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.07349573075771332,
  'sequence': 'Ala ahụ  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 310,
  'token_str': ' ahụ'},
 {'score': 0.030463453382253647,
  'sequence': 'Ala Igbo  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 900,
  'token_str': ' Igbo'},
 {'score': 0.02490180917084217,
  'sequence': 'Ala Nigeria  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 1570,
  'token_str': ' Nigeria'},
 {'score': 0.01187092810869217,
  'sequence': 'Ala ọrụ  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 477,
  'token_str': ' ọrụ'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Ọ bụ <mask>a ka a na- arịa .") #= mmadụ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.044008973985910416,
  'sequence': 'Ọ bụ-a ka a na- arịa.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.0439983569085598,
  'sequence': 'Ọ bụ nwaa ka a na- arịa.',
  'token': 419,
  'token_str': ' nwa'},
 {'score': 0.041657522320747375,
  'sequence': 'Ọ bụ Nwannaa ka a na- arịa.',
  'token': 1459,
  'token_str': ' Nwanna'},
 {'score': 0.018149062991142273,
  'sequence': 'Ọ bụ ọrịaa ka a na- arịa.',
  'token': 956,
  'token_str': ' ọrịa'},
 {'score': 0.009843925014138222,
  'sequence': 'Ọ bụ naanịa ka a na- arịa.',
  'token': 769,
  'token_str': ' naanị'}]

In [ ]:
#mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#move model to gdrive
shutil.move('model path','drive path')

'/content/gdrive/MyDrive/IBO_BETA/IgboBert'